Get API data

In [ ]:
# import libraries
import pandas as pd
import requests
import time

In [ ]:
def get_reputation(item):
    account_id = item['owner']['account_id']
    reputation = item['owner']['reputation']
    return account_id, reputation

In [ ]:
def get_account_rep(df):
    # make API call
    url = "https://api.stackexchange.com/2.3/questions?order=desc&sort=activity&site=stackoverflow" 
    response = requests.get(url).json()
    # wait 1 second before running for loop
    time.sleep(1)

    for item in response['items']:
        account_id, reputation = get_reputation(item)
        # save data in pandas df
        df = df.append({'account_id': account_id, 'reputation': reputation},ignore_index = True)
    return df

In [ ]:
# build our dataframe
df = pd.DataFrame(columns = ['account_id','reputation'])
df = get_account_rep(df)

Import a Pandas DataFrame to a Database in Python

In [ ]:
# Make a connection to database
#https://serverfault.com/questions/1011144/aws-rds-to-pgadmin-error-saving-properties-unable-to-connect-to-server-timeout
! pip install psycopg2
import psycopg2 as ps

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
         conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn 

In [ ]:

host_name = 'database-1.xxxxxxxxxxxxxx.rds.amazonaws.com'
dbname = 'xxxx'
port = '5432'
username = 'postgres'
password = 'xxxxxxx'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)
#also need to check security group rules for inbound/outbound

Connected!


In [ ]:
# create table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS REPUTATION (
    ACCOUNT_ID INTEGER PRIMARY KEY, 
    REPUTATION INTEGER NOT NULL
    )""")  
    curr.execute(create_table_command)

In [ ]:

#check to see if account_id exists
def check_if_account_exists(curr,account_id):
    query = ("""SELECT ACCOUNT_ID FROM REPUTATION WHERE ACCOUNT_ID = %s""")
    curr.execute(query,(account_id,))
    return curr.fetchone() is not None

In [ ]:
#update row if account exists
def update_row(curr, account_id, reputation):
    query = ("""UPDATE REPUTATION
            SET ACCOUNT_ID = %s,
                REPUTATION = %s,
            WHERE ACCOUNT_ID = %s;""")
    vars_to_update = (account_id,reputation)
    curr.execute(query,vars_to_update)

In [ ]:
#updating the database
#handling data for scalability

#check to see if account_id exists
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['account_id','reputation'])
    for i, row in df.iterrows():
        if check_if_account_exists(curr,row['account_id']): # If account already exists then we will update
            update_row(curr,row['account_id'],row['reputation'])
        else: # The account doesn't exist so we will append to the db table
            tmp_df = tmp_df.append(row)
    return tmp_df

#perform update on exisiting account_id
#perform an insert on new videos  

In [ ]:
#write insert command
def insert_into_table(curr,account_id,reputation):
    insert_into_reputation = ("""INSERT INTO REPUTATION (ACCOUNT_ID,REPUTATION) VALUES (%s,%s);""")
    row_to_insert = (account_id,reputation)
    curr.execute(insert_into_reputation,row_to_insert)


In [ ]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['account_id'],row['reputation'])


In [ ]:
curr = conn.cursor()
#conn.rollback
create_table(curr)

conn.commit()

In [ ]:
new_aid_df = update_db(curr,df)

In [ ]:
append_from_df_to_db(curr,new_aid_df)

In [ ]:
conn.commit()